In [1]:
import pandas as pd
import numpy as np
import re
from sklearn import linear_model
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot as plt
from textblob import TextBlob
import statsmodels.api as sm

In [2]:
import import_ipynb
from evaluation import precision
from runTime import runTime

importing Jupyter notebook from evaluation.ipynb
importing Jupyter notebook from runTime.ipynb


In [3]:
df1=pd.read_csv("E:\\5th-Sem\\DMA Project\\Project\\weibo_train1_cpts.csv")
df2=pd.read_csv("E:\\5th-Sem\\DMA Project\\Project\\weibo_train2_cpts.csv")
frames=[df1,df2]
train_all=pd.concat(frames)

In [5]:
train_all.shape[0]

1229618

In [6]:
train=train_all[0:983694]
predict=train_all[983695:1229618]

In [8]:
train.columns

Index(['u_id', 'm_id', 'forward_count', 'comment_count', 'like_count',
       'content', 'date', 'time', 'content_media_count', 'content_#_count',
       'content_@_count', 'content_?_count', 'content_!_count',
       'content_length', 'content_emoji_count', 'hour', 'min', 'sec',
       'forward_min', 'forward_max', 'forward_median', 'forward_mean',
       'comment_min', 'comment_max', 'comment_median', 'comment_mean',
       'like_min', 'like_max', 'like_median', 'like_mean'],
      dtype='object')

# Model 1 - Linear Regression


In [14]:
X_train=train[["content_media_count","content_length","content_emoji_count","hour","min","sec","forward_median","comment_median","like_median"]]
Y_train=train[["forward_count","comment_count","like_count"]]
X_test=predict[["content_media_count","content_length","content_emoji_count","hour","min","sec","forward_median","comment_median","like_median"]]
Y_test=predict[["forward_count","comment_count","like_count"]]
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

pd.options.mode.use_inf_as_na = True
X_train.fillna(X_train.max(),inplace=True)
X_test.fillna(X_test.max(),inplace=True)

lm=linear_model.LinearRegression()
model=lm.fit(X_train,Y_train)
pred1=lm.predict(X_test)


(983694, 9) (983694, 3)
(245923, 9) (245923, 3)


C:\Users\DELL\Anaconda3\lib\site-packages\pandas\core\generic.py:5430: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [15]:
temp = pd.DataFrame.from_records(pred1)
temp=temp.round()
temp=(np.maximum(temp,0))
train_real_pred=Y_test
train_real_pred['fp']=temp[0].values
train_real_pred['cp']=temp[1].values
train_real_pred['lp']=temp[2].values
print("Score:{0:.2f}%".format(precision(train_real_pred.values)*100))

C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#ind

Score:22.83%


In [ ]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=10)
kf.get_n_splits(X)  
KFold(n_splits=10, random_state=None, shuffle=False)
for train_index, test_index in kf.split(X):
print("TRAIN:", train_index, "TEST:", test_index)
X_train, X_test = X[train_index], X[test_index]
y_train, y_test = y[train_index], y[test_index]

# Model 2 - Random Forest

In [9]:
from sklearn.ensemble import RandomForestRegressor

In [111]:
## Spliting of training dataset into 70% training data and 30% testing data randomly
features_train=train[["content_media_count","content_#_count","content_length","content_emoji_count","forward_median","comment_median","like_median"]]
features_test=predict[["content_media_count","content_#_count","content_length","content_emoji_count","forward_median","comment_median","like_median"]]
labels_train=train[['forward_count', 'comment_count', 'like_count']]
labels_test=predict[['forward_count', 'comment_count', 'like_count']]

x = features_train
y = labels_train
x1 = features_test
y1 = labels_test



regr = RandomForestRegressor(max_depth=50, random_state=0,n_estimators=100)
regr.fit(x, y)
pred2 = regr.predict(x1)

In [112]:
temp = pd.DataFrame.from_records(pred2)
temp=temp.round()
temp=(np.maximum(temp,0))
temp=temp.abs()
temp=temp.astype(int)
train_real_pred=Y_test
train_real_pred['fp']=temp[0].values
train_real_pred['cp']=temp[1].values
train_real_pred['lp']=temp[2].values
print("Score:{0:.2f}%".format(precision(train_real_pred.values)*100))

C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.h

Score:30.01%


In [118]:
#train_real_pred

# Model 3 - OLS

In [38]:
model3=sm.OLS(Y_train,X_train).fit()
pred3=model3.predict(X_test)

# Model 4- Ridge

In [39]:
lm1=linear_model.Ridge(alpha=3)
model4=lm1.fit(X_train,Y_train)
pred4=lm1.predict(X_test)

# Model 5- Lasso

In [40]:
lm1=Lasso(alpha=0.01)
model5=lm1.fit(X_train,Y_train)
pred5=lm1.predict(X_test)

# Ensemble - Averaging

In [41]:
pred=(pred1+pred2+pred3+pred4+pred5)/5

In [46]:
pred=pred.round()
pred=(np.maximum(pred,0))

In [58]:
pred=pred.abs()
pred1=pred.astype(int)

In [61]:
pred1

,0,1,2
368886,1,0,0
368887,1,0,0
368888,1,0,0
368889,1,0,0
368890,0,0,0
368891,106,22,91
368892,1,0,0
368893,1,2,2
368894,6,3,3
368895,1,0,0


In [59]:
#np.savetxt("E://DMA_PRED//result_Ensemble.csv",pred1,delimiter=',',header="forward_count,comment_count,like_count",comments="")
#result1=pd.read_csv("E://DMA_PRED//result_Ensemble.csv")

In [63]:
#result1

In [85]:
train_real_pred=Y_test
train_real_pred['fp']=pred1[0]
train_real_pred['cp']=pred1[1]
train_real_pred['lp']=pred1[2]
print("Score:{0:.2f}%".format(precision(train_real_pred.values)*100))

C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the 

Score:24.22%


In [113]:
#train_real_pred

# XG Boost

In [116]:
x = features_train
y = labels_train
x1 = features_test
y1 = labels_test
import xgboost as xgb

C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [119]:
#T_train_xgb = xgb.DMatrix(x, y)
#params = {"objective": "reg:linear", "booster":"gblinear"}
#gbm = xgb.train(dtrain=T_train_xgb,params=params)
#Y_pred = gbm.predict(xgb.DMatrix(x1))
#print(r2_score(y1, Y_pred) )  #xgboost

# Mapping Uid


In [140]:
unique_id=train_all['u_id'].unique().tolist()

In [141]:
uid_df = pd.DataFrame({'u_id':unique_id})

In [143]:
uid_df.shape[0]

37263

In [160]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(unique_id)
#list(le.classes_)


LabelEncoder()

In [161]:
l=[]
l=le.transform(unique_id) 
df = pd.DataFrame({'u_id':l})
uid_df['id']=df['u_id']

In [163]:
train_all=train_all.set_index('u_id').join(uid_df.set_index('u_id'))

In [168]:
train=train_all[0:983694]
predict=train_all[983695:1229618]

In [169]:
X_train=train[["id","content_media_count","content_length","content_emoji_count","hour","min","sec","forward_median","comment_median","like_median"]]
Y_train=train[["forward_count","comment_count","like_count"]]
X_test=predict[["id","content_media_count","content_length","content_emoji_count","hour","min","sec","fo
                rward_median","comment_median","like_median"]]
Y_test=predict[["forward_count","comment_count","like_count"]]
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

pd.options.mode.use_inf_as_na = True
X_train.fillna(X_train.max(),inplace=True)
X_test.fillna(X_test.max(),inplace=True)

lm=linear_model.LinearRegression()
model=lm.fit(X_train,Y_train)
pred1=lm.predict(X_test)


(983694, 10) (983694, 3)
(245923, 10) (245923, 3)


C:\Users\DELL\Anaconda3\lib\site-packages\pandas\core\generic.py:5430: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [170]:
temp = pd.DataFrame.from_records(pred1)
temp=temp.round()
temp=(np.maximum(temp,0))
temp=temp.abs()
temp=temp.astype(int)
train_real_pred=Y_test
train_real_pred['fp']=temp[0].values
train_real_pred['cp']=temp[1].values
train_real_pred['lp']=temp[2].values
print("Score:{0:.2f}%".format(precision(train_real_pred.values)*100))

C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.h

Score:25.69%


In [173]:
## Spliting of training dataset into 70% training data and 30% testing data randomly
features_train=train[["id","content_media_count","content_#_count","content_length","content_emoji_count","forward_median","comment_median","like_median"]]
features_test=predict[["id","content_media_count","content_#_count","content_length","content_emoji_count","forward_median","comment_median","like_median"]]
labels_train=train[['forward_count', 'comment_count', 'like_count']]
labels_test=predict[['forward_count', 'comment_count', 'like_count']]

x = features_train
y = labels_train
x1 = features_test
y1 = labels_test



regr = RandomForestRegressor(max_depth=50, random_state=0,n_estimators=100)
regr.fit(x, y)
pred2 = regr.predict(x1)

In [174]:
temp = pd.DataFrame.from_records(pred2)
temp=temp.round()
temp=(np.maximum(temp,0))
temp=temp.abs()
temp=temp.astype(int)
train_real_pred=Y_test
train_real_pred['fp']=temp[0].values
train_real_pred['cp']=temp[1].values
train_real_pred['lp']=temp[2].values
print("Score:{0:.2f}%".format(precision(train_real_pred.values)*100))

C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.h

Score:20.74%
